In [1]:
datadir = "../data/"
datafilename = "Certainty.xlsx"
datafilepath = datadir + datafilename

In [2]:
from itertools import combinations
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split


# Read in network information from sheet P0
# For now we only use the folloiwng columns in each sheet
cols_used = ['Link ID','ANODE','BNODE','A X_COORD','A Y_COORD','B X_COORD','B Y_COORD','Link Length(miles)','# of lanes-A','Capacity-A (veh/h)',
            'auto volume(2010)-A','AADT(2010)-A','Speed(mph)-A','VMT-A']
# # For now we assume all occurrences of the same link in the same sheet share the same attribute values
df_p0= pd.read_excel(datafilepath, sheet_name='P0',usecols=cols_used).dropna(subset=['Link ID'])
# df_p0_unique = df_p0.drop_duplicates(subset=['Link ID']).dropna(subset=['Link ID', 'A X_COORD', 'A Y_COORD', 'B X_COORD', 'B Y_COORD'])


In [3]:
### Only Read in Project 3, 4 and 5
integers = [ 3, 4, 5]
def generate_combinations(integers):
    all_combinations = []
    # Loop through lengths from 1 to 6
    for length in range(1, 1+len(integers)):
        # Generate combinations of the current length
        comb = combinations(integers, length)
        # Convert each combination to a string, add "p", and add to the list
        all_combinations.extend(['P' + ''.join(map(str, c)) for c in comb])
    return all_combinations
# Generate all combinations
sheet_names = generate_combinations(integers)

# Insert "P0" at the beginning of the list
sheet_names.insert(0, "P0")


In [ ]:
##### Read in all 2^6 sheets of data ###


# ## Create a list of sheet names ##

# # List of the first 6 integers
# integers = [1, 2, 3, 4, 5, 6]

# # Function to generate combinations
# def generate_combinations(integers):
#     all_combinations = []
#     # Loop through lengths from 1 to 6
#     for length in range(1, 7):
#         # Generate combinations of the current length
#         comb = combinations(integers, length)
#         # Convert each combination to a string, add "p", and add to the list
#         all_combinations.extend(['P' + ''.join(map(str, c)) for c in comb])
#     return all_combinations

# # Generate all combinations
# sheet_names = generate_combinations(integers)

# # Insert "P0" at the beginning of the list
# sheet_names.insert(0, "P0")

In [4]:
# Preparing the sample set
dataset = []
for ii in range(len(sheet_names)):
    # read the correct sheet 
    df= pd.read_excel(datafilepath, sheet_name=sheet_names[ii],usecols=cols_used).dropna(subset=['Link ID'])
    
    # dataframe for edge attributes
    df_edge_attr= np.column_stack((df['Link Length(miles)'],df['# of lanes-A'],df['Capacity-A (veh/h)'], df['Speed(mph)-A'],df['auto volume(2010)-A']))
    # edge_attr = torch.tensor(df_edge_attr, dtype=torch.float)

    df_edge_labels = df['AADT(2010)-A'].to_numpy()
    # edge_labels = torch.tensor(df['AADT(2010)-A'].to_numpy(),dtype=torch.float)
    # new_data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=edge_labels)
    
    sample = (df_edge_attr, df_edge_labels)
    dataset.append(sample)




In [5]:
X = [sample[0] for sample in dataset]
y = [sample[1] for sample in dataset]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)




# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float)
y_train = torch.tensor(y_train,dtype=torch.float)
X_val = torch.tensor(X_val,dtype=torch.float)
y_val = torch.tensor(y_val,dtype=torch.float)


/tmp/ipykernel_258328/3189767075.py:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  X_train = torch.tensor(X_train, dtype=torch.float)


In [6]:
 
# Define the neural network model
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(5, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = RegressionModel()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 100
batch_size = 1

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    for i in range(X_train.shape[0]):
        optimizer.zero_grad()
        outputs = model(X_train[i])
        loss = criterion(outputs.squeeze(), y_train[i])
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {format(epoch_loss/X_train.shape[0],".2e")}')

# Evaluate the model
model.eval()
val_loss = 0.0
relative_val_loss = 0.0
sum_edge_labels = 0.0
with torch.no_grad():
    for i in range(X_val.shape[0]):
        outputs = model(X_val[i])
        loss = criterion(outputs.squeeze(), y_val[i])
        val_loss += loss.item()
        
        # Calculate Relative Validation Loss
        edge_labels_tensor = criterion(torch.zeros(torch.Tensor.size(y_val[i])), y_val[i])
        sum_edge_labels += edge_labels_tensor.item()
        relative_loss = loss/edge_labels_tensor
        relative_val_loss += relative_loss.item()
print(f'Absolute Validation Loss: {format(val_loss/X_val.shape[0],".2e")}')

# calculate average edge_labels_tensor.item()
ave_edge_labels = sum_edge_labels/X_val.shape[0]
print("\n Relative Loss = ", format(val_loss/X_val.shape[0], '.2e'), " / ", format(ave_edge_labels,'.2e'), " = ", format(relative_val_loss/X_val.shape[0],'.2e'), "\n")
 

# Make predictions
model.eval()
with torch.no_grad():
    for i in range(X_val.shape[0]):
        predictions = model(X_val[i]).squeeze()
        print(f'Sample {i+1} Predictions:\n', predictions)

/home/duantu/soft/transportation_venv/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Epoch 1/100, Loss: 4.69e+09
Epoch 2/100, Loss: 4.26e+09
Epoch 3/100, Loss: 3.84e+09
Epoch 4/100, Loss: 3.38e+09
Epoch 5/100, Loss: 2.88e+09
Epoch 6/100, Loss: 2.37e+09
Epoch 7/100, Loss: 1.85e+09
Epoch 8/100, Loss: 1.34e+09
Epoch 9/100, Loss: 8.84e+08
Epoch 10/100, Loss: 5.15e+08
Epoch 11/100, Loss: 2.67e+08
Epoch 12/100, Loss: 1.45e+08
Epoch 13/100, Loss: 1.18e+08
Epoch 14/100, Loss: 1.27e+08
Epoch 15/100, Loss: 1.29e+08
Epoch 16/100, Loss: 1.22e+08
Epoch 17/100, Loss: 1.15e+08
Epoch 18/100, Loss: 1.13e+08
Epoch 19/100, Loss: 1.13e+08
Epoch 20/100, Loss: 1.13e+08
Epoch 21/100, Loss: 1.13e+08
Epoch 22/100, Loss: 1.13e+08
Epoch 23/100, Loss: 1.12e+08
Epoch 24/100, Loss: 1.12e+08
Epoch 25/100, Loss: 1.12e+08
Epoch 26/100, Loss: 1.11e+08
Epoch 27/100, Loss: 1.11e+08
Epoch 28/100, Loss: 1.11e+08
Epoch 29/100, Loss: 1.11e+08
Epoch 30/100, Loss: 1.10e+08
Epoch 31/100, Loss: 1.10e+08
Epoch 32/100, Loss: 1.10e+08
Epoch 33/100, Loss: 1.10e+08
Epoch 34/100, Loss: 1.10e+08
Epoch 35/100, Loss: 1.1